In [1]:
import json
import csv
import os.path
from copy import copy
import numpy as np
import numpy.linalg as lin
import pandas as pd
import requests
from pprint import pprint as pp
import matplotlib.pyplot as plt
import math

from collections import Counter

import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split


import seaborn as sns
from scipy.stats import pearsonr

from sklearn.manifold import TSNE

%matplotlib inline
%config InlineBackend.figure_format='retina'

np.set_printoptions(threshold=np.inf, linewidth=np.inf)
pd.set_option('display.max_rows',10000)

In [2]:
with open('./merge_json_recipes_JV.json', encoding='UTF-8') as all_data:
    all_data_raw = json.load(all_data)


In [3]:
all_data = all_data_raw.copy()

In [4]:
remove_list_1=['KRICT-SC-00366',
               'KRICT-SC-00644',
               'KRICT-SC-00645',
               'KRICT-SC-00646',
               'KRICT-SC-00647',
               'KRICT-SC-00648',
               'KRICT-SC-00649',
               'KRICT-SC-00650',
               'KRICT-SC-00651',
               'KRICT-SC-00652',
               'KRICT-SC-00653',
               'KRICT-SC-00654',
               'KRICT-SC-00655',
               'KRICT-SC-00656',
               'KRICT-SC-00689',
               'KRICT-SC-00690',
               'SC-00445',
               'SC-00525',
               'CNU_EEML-SC-00057',
               'CNU_EEML-SC-00215',
               'CNU_EEML-SC-00242',
               'CNU_EEML-SC-00249',
               'CNU_EEML-SC-00256',
               'CNU_EEML-SC-00262',
               'CNU_EEML-SC-00263',
               'CNU_EEML-SC-00295',
               'CNU_EEML-SC-00327',
               'CNU_EEML-SC-00385',
               'CAU-SC-00269',
               'CAU-SC-00380',
               'CAU-SC-00792',
               'CAU-SC-00880',
               'KNU-SC-00120',
               'CNU_HEMD-SC-00296',
               'CNU_HEMD-SC-00297',
               'CNU_HEMD-SC-00298',
               'CNU_HEMD-SC-00299',
               'CNU_HEMD-SC-00300',
               'CNU_HEMD-SC-00301',
               'CNU_HEMD-SC-00302',
               'CNU_HEMD-SC-00303',
               'CNU_HEMD-SC-00304',
               'CNU_HEMD-SC-00305',
               'CNU_HEMD-SC-00306',
               'CNU_HEMD-SC-00307',
               'CNU_HEMD-SC-00308',
               'CNU_HEMD-SC-00309',
               'CNU_HEMD-SC-00310',
               'CNU_HEMD-SC-00311',
               'CNU_HEMD-SC-00312',
               'CNU_HEMD-SC-00313',
               'SC-00853',
               'SC-00860',
               'SC-00861'               # These data contain some undefined erroros.
               ]
              
author_list = {
    36: "김범수",
    38: "Sae_Jin_Sung",
    39: "김성후",
    45: "이진원",
    49: "홍순일",
    50: "유소민",
    51: "박은영",
    56: "문수지",
    64: "배미선",
    85: "유수빈",
    90: "오진호",
    125: "이경민",
    131: "Passarut_Boonmongkolras",
    133: "주연경",
    11: "Ji_Sang-Geun",
    129: "김유영",
    24: "Noh_Eunseo",
    16: "Manman_Hu",
    6: "Hyeonwoo_Kim",
    10:"Jin_Wook_Yoo",
    30:"김종범",
    9: "Gwisu_Kim",
    134: "김준석",
    43: "Seung-Woo Kim",
    86: "전효경",
    58: "HyunSung Yun",
    65: "채은총",
    142: "김수아",
    140: "PARK SUNG HWAN",
    136: "홍민정",
    135: "Park Yerang",
    132: "Gill Sang Han",
    130: "백명철",    129: "김유영",    113: "신나혜",  41: "Yangjeong Park", 40: "Andi Muhammad Risqi",  39: "Sung Hu Kim",   34: "김재희",   31:"Jaewang Park", 30: "김종범",  25: "Hanul Min",   24: "Noh Eunseo", 18: "이승운",  17: "Kwon Hyoungwoo", 16:"Manman Hu", 15: "홍승택", 14:"윤현성",
    12: "백민재", 11:"Ji Ang-Geun",  10: "Jin Wook Yoo", 9: "Gwisu Kim", 8:"LeeDoyoon", 7: "윤소미",  6:"Hyeonwoo Kim", 5:"이승운", #UNIST
    127: "이재희",    126: "박성준",    124: "민경빈",    123: "이현준",    122: "김명승",    121: "권나윤",    120:"김범진",   106: "석승재",   105: "손태웅",  66: "김현서",     #KAIST
    118: "임정인",    112: "김희수",      #CNU_HEMD
    114: "김문회",    79: "신재관",   78: "김미정",   77: "Gisung Kim",   74:"JungYup Yang",    #KNU
    111: "Jin-Wook Lee",    104: "Jin-Soo Kim",    103: "Sang-Hyun Chin",     102: "GwangHee Lee",    101: "ZHANGDONGBO",     100: "Keonwoo Park", 99: "leeseoryeong",   97: "Sung-Kwang Jung",    95: "Zheng WENTING",  94:"GaYeong Kim",    93: "Jae_Hwan Kim",  82: "seunggu choi",  #SKKU
    110: "lee sang won",  109: "Sangheon Lee",    108:  "김창용",   #KU_SECL
    107: "김수민",   73: "조성원",   71: "이동건",   70: "Jitendra bahadur", 69: "이정연",  68: "이서준",   67: "SAEMON YOON",    63: "류준",             #CAU
    98: "권성남", 97: "나석인",      #JBNU
    92: "김수아",    91: "정희윤",     62: "정성진", 61:"양태열",                            #CNU_EEML
}

#all_data[2867]


In [5]:

rm_list = [0, 'none', 'None', None, '', 'NONE','XXXX', 'Material', 'NOne', '0']

def process_material(target, data_input, id):
    material_index = -1  # Default value indicating not found
    material_arr = []
    material_conc_arr = []
    material_quantity_arr = []
    material_SVT_arr = []
    material_SVT_volume_arr = []
    material_spin_arr = []
    material_spinrpm_arr = []
    material_spintime_arr = []
    material_anntemp_arr = []
    material_anntime_arr = []
    material_evap_thick_arr = []
    material_evap_rate_arr = []
    material_evap_pressure_arr = []
    material_atmostype_arr=[]
    material_atmostemp_arr=[]
    material_atmoshumidity_arr=[]
    material_additive_arr=[]
    material_additive_SVT_arr=[]
    material_additive_conc_arr=[]
    material_additive_conc_unit_arr=[]
    material_additive_quantity_arr=[]
    material_cbd_temp_arr=[]
    material_cbd_time_arr=[]
    material_cbd_arr=[]
    material_cbd_quantity_arr=[]
    a_site_arr=[]
    a_site_ratio_arr=[]
    b_site_arr=[]
    b_site_ratio_arr=[]
    x_site_arr=[]
    x_site_ratio_arr=[]
    
    for j, data in enumerate(data_input):
        if target in data_input[j]:
            material_index = j
            material_info = data_input[material_index][target]   
            
            for info in material_info:
                     
                material='' if info['material']['value'] in rm_list else info['material']['value']
                material_arr.append(material) 
                    
                conc = '' if info['concentration']['value'] in rm_list else info['concentration']['value']
                material_conc_arr.append(conc)
    
                quantity = '' if info['quantity']['value'] in rm_list else info['quantity']['value']
                material_quantity_arr.append(quantity)


            if 'solvent' in data:
                for solvent in data['solvent']:
                    material_SVT='' if solvent['material']['value'] in rm_list else solvent['material']['value']
                    material_SVT_arr.append(material_SVT) 
                    material_SVT_volume='' if solvent['volume']['value'] in rm_list else solvent['volume']['value']
                    material_SVT_volume_arr.append(material_SVT_volume)

            if 'spin_coating' in data:
                for spin in data['spin_coating']:
                    material_spin_arr.append(spin.get('type', {}).get('value', ''))
                    material_spinrpm_arr.append(spin['rpm']['value'])
                    material_spintime_arr.append(spin['time']['value'])

            if 'annealing' in data:
                for anneal in data['annealing']:
                    material_anntemp_arr.append(anneal['temperature']['value'])
                    material_anntime_arr.append(anneal['time']['value'])

            if 'thermal_evaporation' in data:
                for evap in data['thermal_evaporation']:
                    material_evap_thick_arr.append(evap['vacuum_evaporation_thickness']['value'])
                    material_evap_rate_arr.append(evap['evaporation_rate']['value'])
                    material_evap_pressure_arr.append(evap['vacuum_pressure']['value'])

            if 'chemical_bath_deposition' in data:
                
                cbd=data['chemical_bath_deposition']
                if 'bath_annealing' in cbd:
                    for kk,bath in enumerate(cbd['bath_annealing']):
                        try:
                            material_cbd_temp_arr.append(bath['temperature']['value'])
                            material_cbd_time_arr.append(bath['time']['value'])
                        except KeyError:
                            continue
                if 'solution' in cbd:
                    for kk,solution in enumerate(cbd['solution']):
                        try:
                            material_cbd_arr.append(solution['material']['value'])
                            material_cbd_quantity_arr.append(solution['quantity']['value'])
                        except KeyError:
                            continue                

            if 'sputtering' in data:
                for sputter in data['sputtering']:
                    print(id, sputter)    
    
    return {
        'material_arr': material_arr,
        'material_conc_arr': material_conc_arr,
        'material_quantity_arr': material_quantity_arr,
        'material_SVT_arr': material_SVT_arr,
        'material_SVT_volume_arr': material_SVT_volume_arr,
        'material_spin_arr': material_spin_arr,
        'material_spinrpm_arr': material_spinrpm_arr,
        'material_spintime_arr': material_spintime_arr,
        'material_anntemp_arr': material_anntemp_arr,
        'material_anntime_arr': material_anntime_arr,
        'material_evap_thick_arr': material_evap_thick_arr,
        'material_evap_rate_arr': material_evap_rate_arr,
        'material_evap_pressure_arr': material_evap_pressure_arr,
        'material_cbd_temp_arr': material_cbd_temp_arr,
        'material_cbd_time_arr': material_cbd_time_arr,
        'material_cbd_arr': material_cbd_arr,
        'material_cbd_quantity_arr': material_cbd_quantity_arr,
        'material_index': material_index
    }

def process_arrays(array_dicts, prefixes, new_row, rm_list, max_lengths):
    for prefix, array_dict in zip(prefixes, array_dicts):
        max_length = 0
        count=0
        
        # Iterate over the zipped arrays
        for j, values in enumerate(zip(*array_dict['arrays'])):
            values = list(values)
           

            # Replace NaN values to the right of numbers with ''
            for i in range(0, len(values)-1):
                
                if values[i] in rm_list and (values[i+1] not in rm_list or (i < len(values) - 2 and values[i+2] not in rm_list)):
                    values[i] = ''       
                elif values[i] in rm_list and values[i+1] in rm_list :
                    values[i] = 'none'
            if values[len(values)-1] in rm_list:
                values[len(values)-1] = 'none'
            
            # Remove values that are in rm_list
            values = [value for value in values if value not in rm_list or value == '']
            
           
            # Update new_row dictionary with valid values
            for k, value in enumerate(values):
                key = f'{prefix}_{j+1}' if (k == 0 and array_dict["names"][k] in rm_list) else f'{prefix}_{array_dict["names"][k]}_{j+1}'
                new_row[key] = value
                if k==0:
                    if (value not in rm_list):
                        count+=1
                
            max_length = max(max_length, count)   
            
        
        # Update max_lengths dictionary with the maximum length found
        max_lengths[prefix] = max(max_lengths.get(prefix, 0), max_length)
        
        
    return

def create_dict(prefix, arrays, names):
    return {"prefix": prefix, "arrays": arrays, "names": names}

def update_max_lists(max_lengths, max_lists):
    for key in max_lengths:
        if key in max_lists:
            max_lists[key] = max(max_lists[key], max_lengths[key])
        else:
            max_lists[key] = max_lengths[key]

In [6]:
def process_pv(target, data_input, id):
    material_index = -1  # Default value indicating not found
    material_arr = []
    material_conc_arr = []
    material_quantity_arr = []
    material_SVT_arr = []
    material_SVT_volume_arr = []
    material_spin_arr = []
    material_spinrpm_arr = []
    material_spintime_arr = []
    material_anntemp_arr = []
    material_anntime_arr = []
    material_evap_thick_arr = []
    material_evap_rate_arr = []
    material_evap_pressure_arr = []
    material_atmostype_arr=[]
    material_atmostemp_arr=[]
    material_atmoshumidity_arr=[]
    material_additive_arr=[]
    material_additive_SVT_arr=[]
    material_additive_conc_arr=[]
    material_additive_conc_unit_arr=[]
    material_additive_quantity_arr=[]
    material_antisolvent_arr=[]
    material_antisolvent_quantity_arr=[]    
    material_antisolvent_dripping_time_arr=[]    
    a_site_arr=[]
    a_site_ratio_arr=[]
    b_site_arr=[]
    b_site_ratio_arr=[]
    x_site_arr=[]
    x_site_ratio_arr=[]
    method_arr=[]
    molarity_arr=[]
    
    
    for j, data in enumerate(data_input):
        if target in data_input[j]:
            material_index = j
            material_info = data_input[material_index][target]   

            if 'method' in material_info:
                method_arr.append(material_info['method']['value'])

            if 'molarity' in material_info:
                molarity_arr.append(material_info['molarity']['value'])
                
                
                
            if 'a_site' in material_info:

                for a_site in material_info['a_site']:

                    try:
                        if float(a_site['ratio']['value']) > 0.0:
                            material='' if a_site['material']['value'] in rm_list else a_site['material']['value']
                            a_site_arr.append(material) 
                            a_site_ratio_arr.append(a_site['ratio']['value'])

                    except:
                        print('a_site ratio string', id)

            if 'b_site' in material_info:                
                for b_site in material_info['b_site']:

                    try:
                        if float(b_site['ratio']['value']) > 0.0:
                            material='' if b_site['material']['value'] in rm_list else b_site['material']['value']
                            b_site_arr.append(material) 
                            b_site_ratio_arr.append(b_site['ratio']['value'])

                    except:
                        print('b_site ratio string', id)

            if 'x_site' in material_info:                
                for x_site in material_info['x_site']:

                    try:
                        if float(x_site['ratio']['value']) > 0.0:
                            material='' if x_site['material']['value'] in rm_list else x_site['material']['value']
                            x_site_arr.append(material) 
                            x_site_ratio_arr.append(x_site['ratio']['value'])
                            
                    except:
                        print('x_site ratio string', id)

                        
            if 'antisolvent' in material_info:
                for antisolvent in material_info['antisolvent']:
                    material_antisolvent='' if antisolvent['material']['value'] in rm_list else antisolvent['material']['value']
                    material_antisolvent_arr.append(material_antisolvent)
                    material_antisolvent_quantity_arr.append(antisolvent['quantity']['value'])
                    material_antisolvent_dripping_time_arr.append(antisolvent['dripping_time']['value'])

            if 'solvent' in material_info:
                for solvent in material_info['solvent']:
                    material_SVT='' if solvent['material']['value'] in rm_list else solvent['material']['value']
                    material_SVT_arr.append(material_SVT) 
                    material_SVT_volume='' if solvent['volume']['value'] in rm_list else solvent['volume']['value']
                    material_SVT_volume_arr.append(material_SVT_volume)

            if 'spin_coating' in material_info:
                for spin in material_info['spin_coating']:
                    material_spin_arr.append(spin.get('type', {}).get('value', ''))
                    material_spinrpm_arr.append(spin['rpm']['value'])
                    material_spintime_arr.append(spin['time']['value'])

            if 'annealing' in material_info:
                for anneal in material_info['annealing']:
                    material_anntemp_arr.append(anneal['temperature']['value'])
                    material_anntime_arr.append(anneal['time']['value'])

            if 'thermal_evaporation' in material_info:
                for evap in material_info['thermal_evaporation']:
                    material_evap_thick_arr.append(evap['vacuum_evaporation_thickness']['value'])
                    material_evap_rate_arr.append(evap['evaporation_rate']['value'])
                    material_evap_pressure_arr.append(evap['vacuum_pressure']['value'])

            if 'atmosphere' in material_info:
                for atmosphere in material_info['atmosphere']:
                    material_atmostype_arr.append(atmosphere['type']['value'])
                    material_atmostemp_arr.append(atmosphere['temperature']['value'])    
                    material_atmoshumidity_arr.append(atmosphere['humidity']['value'])    
                    
            if 'additive' in material_info:
                for additive in material_info['additive']:
                    material_additive='' if additive['material']['value'] in rm_list and additive['quantity']['value'] in rm_list else additive['material']['value']
                    material_additive_arr.append(material_additive)
                    material_additive_quantity='' if additive['quantity']['value'] in rm_list else additive['quantity']['value']
                    material_additive_quantity_arr.append(material_additive_quantity)
                    try:
                        material_additive_SVT_arr.append(additive['solvent']['value'])    
                        material_additive_conc_arr.append(additive['concentration']['value'])
                        material_additive_conc_unit_arr.append(additive['conc_unit']['value'])
                    except:
                        print('additive old version', id)

            # Check if all elements are less than 1 (error correction from 1 to 100%)
            def process_list_of_strings(list_of_strings):
                try:
                    # convert string to number
                    numeric_list = [float(x) for x in list_of_strings]
                except:
                    numeric_list = list_of_strings
                    
                # Check if all elements are less than 1.
                if all(x < 1 for x in numeric_list):
                    # multiply by 100
                    numeric_list = [x * 100 for x in numeric_list]
                    print('correct the PV ratio', id)
                
                return numeric_list
            
            a_site_ratio_arr = process_list_of_strings(a_site_ratio_arr)
            b_site_ratio_arr = process_list_of_strings(b_site_ratio_arr)
            x_site_ratio_arr = process_list_of_strings(x_site_ratio_arr)

    
    return {
        'material_SVT_arr': material_SVT_arr,
        'material_SVT_volume_arr': material_SVT_volume_arr,
        'material_spin_arr': material_spin_arr,
        'material_spinrpm_arr': material_spinrpm_arr,
        'material_spintime_arr': material_spintime_arr,
        'material_anntemp_arr': material_anntemp_arr,
        'material_anntime_arr': material_anntime_arr,
        'material_evap_thick_arr': material_evap_thick_arr,
        'material_evap_rate_arr': material_evap_rate_arr,
        'material_evap_pressure_arr': material_evap_pressure_arr,
        'material_atmostype_arr': material_atmostype_arr,
        'material_atmostemp_arr': material_atmostemp_arr,
        'material_atmoshumidity_arr': material_atmoshumidity_arr,
        'material_additive_arr': material_additive_arr,
        'material_additive_SVT_arr': material_additive_SVT_arr,
        'material_additive_conc_arr': material_additive_conc_arr,
        'material_additive_conc_unit_arr': material_additive_conc_unit_arr,
        'material_additive_quantity_arr': material_additive_quantity_arr,
        'material_antisolvent_arr': material_antisolvent_arr,
        'material_antisolvent_quantity_arr':material_antisolvent_quantity_arr,
        'material_antisolvent_dripping_time_arr': material_antisolvent_dripping_time_arr,
        'a_site_arr':a_site_arr,
        'a_site_ratio_arr':a_site_ratio_arr,
        'b_site_arr':b_site_arr,
        'b_site_ratio_arr':b_site_ratio_arr,
        'x_site_arr':x_site_arr,
        'x_site_ratio_arr':x_site_ratio_arr,
        'PV_molarity_arr':molarity_arr,
        'PV_method_arr':method_arr,
        'material_index': material_index
    }

In [7]:
df = pd.DataFrame(columns=['id','TCO'])
rows=list()



max_lists = {
    'BL': 0, 'BL_SVT': 0, 'BL_spin': 0, 'BL_ann': 0, 'BL_evap': 0, 'BL_cbd_bath':0, 'BL_cbd_solution':0,
    'ETL': 0, 'ETL_SVT': 0, 'ETL_spin': 0, 'ETL_ann': 0, 'ETL_evap': 0, 'ETL_cbd_bath':0, 'ETL_cbd_solution':0, 'ETL_doping': 0,
    'ETL_treatment': 0, 'ETL_treatment_SVT': 0, 'ETL_treatment_spin': 0, 'ETL_treatment_ann': 0, 'ETL_treatment_evap': 0,
    'ETL_treatment_cbd_bath':0, 'ETL_treatment_cbd_solution':0,
    'HTL': 0, 'HTL_SVT': 0, 'HTL_spin': 0, 'HTL_ann': 0, 'HTL_evap': 0, 'HTL_cbd_bath':0, 'HTL_cbd_solution':0, 'HTL_doping': 0,
    'HTL_treatment': 0, 'HTL_treatment_SVT': 0, 'HTL_treatment_spin': 0, 'HTL_treatment_ann': 0, 'HTL_treatment_evap': 0,
    'HTL_treatment_cbd_bath':0, 'HTL_treatment_cbd_solution':0,
    'PV_STR': 0, 'PV_STR_SVT': 0, 'PV_STR_spin': 0, 'PV_STR_ann': 0, 'PV_STR_evap': 0,
    'PV_STR_cbd_bath':0, 'PV_STR_cbd_solution':0,
    'A_site':0, 'B_site':0, 'X_site':0, 'SVT_max':0, 'PV_spin_max':0, 'PV_ann_max':0, 'PV_evap_max':0, 'PV_atmos_max':0,
    'ADD_max':0, 'ASV_max':0
}

Err_num=0



materials_to_process = ['blocking_buffer_layer', 'etl', 'etl_treatment', 'htl', 'htl_treatment', 'perovskite_treatment']
results={}

for i in range(len(all_data)):

    if (all_data[i]['id'] not in remove_list_1):
        
        #author
        author = author_list.get(all_data[i]['author'])
        
        #TCO
        TCO_index = 100
        for j in range(len(all_data[i]['input'])):
            if ('tco' in all_data[i]['input'][j]):
                TCO_index = j
                tco = all_data[i]['input'][TCO_index]['tco']['value']
                

        data_input = all_data[i]['input']
        id = all_data[i]['id']
        
        for material in materials_to_process:
            results[material] = process_material(material, data_input, id)        

        BL_results = results['blocking_buffer_layer']
        ETL_results = results['etl']
        ETL_treatment_results = results['etl_treatment']
        HTL_results = results['htl']
        HTL_treatment_results = results['htl_treatment']
        PV_STR_results = results['perovskite_treatment']
        

        PV_results=process_pv('perovskite',data_input,id)
        
        
        #ETL_doping
        ETL_doping_arr=[]
        ETL_doping_volume_arr=[]
        ETL_doping_index=100
        for j in range(len(all_data[i]['input'])):
            if ('etl_doping' in all_data[i]['input'][j]):
                ETL_doping_index=j
                for k in range(len(all_data[i]['input'][ETL_doping_index]['etl_doping'])):
                    ETL_doping_arr.append(all_data[i]['input'][ETL_doping_index]['etl_doping'][k]['material']['value'])
                    ETL_doping_volume_arr.append(all_data[i]['input'][ETL_doping_index]['etl_doping'][k]['volume']['value'])
  
        
 
        
        #HTL_doping
        HTL_doping_arr=[]
        HTL_doping_volume_arr=[]
        HTL_doping_index=100
        for j in range(len(all_data[i]['input'])):
            if ('htl_doping' in all_data[i]['input'][j]):
                HTL_doping_index=j
                for k in range(len(all_data[i]['input'][HTL_doping_index]['htl_doping'])):
                    HTL_doping_arr.append(all_data[i]['input'][HTL_doping_index]['htl_doping'][k]['material']['value'])
                    HTL_doping_volume_arr.append(all_data[i]['input'][HTL_doping_index]['htl_doping'][k]['volume']['value'])

            
                    
        #Anode
        Anode_index = 100
        
        for j in range(len(all_data[i]['input'])):
            if ('anode_metal' in all_data[i]['input'][j]):
                Anode_index = j
                AND=all_data[i]['input'][Anode_index]['anode_metal']['value']
                #print(i, all_data[i]['id'],HTL_index,len(all_data[i]['input'][HTL_index]))


                
        #Note
        note_index = 100
        for j in range(len(all_data[i]['input'])):
            if ('note' in all_data[i]['input'][j]):
                note_index = j
                note=all_data[i]['input'][note_index]['note']['value']


        device_type_index = 100
        device_type=None
        for j in range(len(all_data[i]['input'])):
            if ('device_type' in all_data[i]['input'][j]):
                device_type_index = j        
                device_type=all_data[i]['input'][device_type_index]['device_type']['value']
                #print(i, all_data[i]['id'],device_type, len(all_data[i]['input'][device_type_index]))

        type_index = 100
        for j in range(len(all_data[i]['input'])):
            if ('type' in all_data[i]['input'][j]):
                type_index = j        
                type=all_data[i]['input'][type_index]['type']['value']
                if type == 'p-i-n' or type == 'n-i-p':
                    temp = device_type
                    device_type = type
                    type = device_type
                    device_type_index = 1

        temperature_index = 100
        temperature=None
        for j in range(len(all_data[i]['input'])):
            if ('temperature' in all_data[i]['input'][j]):
                temperature_index = j        
                temperature=all_data[i]['input'][temperature_index]['temperature']['value']

        humidity_index = 100
        humidity=None
        for j in range(len(all_data[i]['input'])):
            if ('humidity' in all_data[i]['input'][j]):
                humidity_index = j        
                humidity=all_data[i]['input'][humidity_index]['humidity']['value']        

        cell_dimension_index = 100
        cell_dimension=None
        for j in range(len(all_data[i]['input'])):
            if ('cell_dimension' in all_data[i]['input'][j]):
                cell_dimension_index = j        
                cell_dimension=all_data[i]['input'][cell_dimension_index]['cell_dimension']['value']           
        #JV
        Ef_list = []
        FF_list = []
        JS_list = []
        VO_list = []
        Ef_max = 0.0
        Ef_avg = 0.0
        FF_max = 0.0
        FF_avg = 0.0
        JS_max = 0.0
        JS_avg = 0.0
        VO_max = 0.0
        VO_avg = 0.0
        for j in range(len(all_data[i]['JV'])):
            if ('pv_cell_parameters' in all_data[i]['JV'][0]):
                Ef_list.append(all_data[i]['JV'][j]['pv_cell_parameters']['efficiency']['value'])
                FF_list.append(all_data[i]['JV'][j]['pv_cell_parameters']['fill_factor']['value'])
                JS_list.append(all_data[i]['JV'][j]['pv_cell_parameters']['jsc']['value'])
                VO_list.append(all_data[i]['JV'][j]['pv_cell_parameters']['voc']['value'])
            else:
                Ef_list.append(all_data[i]['JV'][j]['efficiency']['value'])
                FF_list.append(all_data[i]['JV'][j]['fill_factor']['value'])
                JS_list.append(all_data[i]['JV'][j]['jsc']['value'])
                VO_list.append(all_data[i]['JV'][j]['voc']['value'])

 
        Ef_list=np.array(Ef_list)
        FF_list=np.array(FF_list)
        JS_list=np.array(JS_list)
        VO_list=np.array(VO_list)
        Ef_list=np.float32(Ef_list)
        FF_list=np.float32(FF_list)
        VO_list=np.float32(VO_list)
        JS_list=np.float32(JS_list)
        Ef_list[np.isnan(Ef_list)] = 0
        FF_list[np.isnan(FF_list)] = 0
        JS_list[np.isnan(JS_list)] = 0
        VO_list[np.isnan(VO_list)] = 0
        
        if (len(Ef_list) > 0):
            Ef_max = Ef_list[np.argmax(Ef_list)]
            Ef_avg = np.average(Ef_list)
            FF_max = FF_list[np.argmax(Ef_list)]
            FF_avg = np.average(FF_list)
            JS_max = JS_list[np.argmax(Ef_list)]
            JS_avg = np.average(JS_list)
            VO_max = VO_list[np.argmax(Ef_list)]
            VO_avg = np.average(VO_list)
        
        if ((TCO_index == 100) or (ETL_results['material_index'] == 100) or (PV_results['material_index'] == 100) or (HTL_results['material_index'] == 100) or (Anode_index == 100) or (Ef_max == 0.0) or (device_type_index == 100)):   
            Err_num = Err_num + 1
            print(all_data[i]['id'], author,'TCO', TCO_index, 'ETL', ETL_results['material_index'], 'PV', PV_results['material_index'], 'HTL', HTL_results['material_index'], 
                  'Anode', Anode_index, 'device', device_type_index)
        else:    
            
            new_row = {'id': all_data[i]['id'], 'author': author, 'note': note, 'TCO': tco, 'AND': AND, 'device_type': device_type,\
                       'Ef_max': Ef_max, 'FF_max':FF_max, 'JS_max': JS_max, 'V0_max':VO_max, 'temperature': temperature, 'humidity': humidity, 'cell_dimension': cell_dimension}
   
            array_dicts = [
                create_dict('BL', [BL_results['material_arr'], BL_results['material_conc_arr'], BL_results['material_quantity_arr']], ["", "conc", "quantity"]),
                create_dict('BL_SVT', [BL_results['material_SVT_arr'], BL_results['material_SVT_volume_arr']], ["", "volume"]),
                create_dict('BL_spin', [BL_results['material_spin_arr'], BL_results['material_spinrpm_arr'], BL_results['material_spintime_arr']], ["", "rpm", "timevalue"]),
                create_dict('BL_ann', [BL_results['material_anntemp_arr'], BL_results['material_anntime_arr']], ["", "timevalue"]),
                create_dict('BL_evap', [BL_results['material_evap_thick_arr'], BL_results['material_evap_rate_arr'], BL_results['material_evap_pressure_arr']], ["thickness", "rate", "pressure"]),
                create_dict('BL_cbd_bath', [BL_results['material_cbd_temp_arr'], BL_results['material_cbd_time_arr']], ["temperature", "time"]),
                create_dict('BL_cbd_solution',[BL_results['material_cbd_arr'], BL_results['material_cbd_quantity_arr']], ["","quantity"]),
                create_dict('ETL', [ETL_results['material_arr'], ETL_results['material_conc_arr'], ETL_results['material_quantity_arr']], ["", "conc", "quantity"]),
                create_dict('ETL_SVT', [ETL_results['material_SVT_arr'], ETL_results['material_SVT_volume_arr']], ["", "volume"]),
                create_dict('ETL_spin', [ETL_results['material_spin_arr'], ETL_results['material_spinrpm_arr'], ETL_results['material_spintime_arr']], ["", "rpm", "timevalue"]),
                create_dict('ETL_ann', [ETL_results['material_anntemp_arr'], ETL_results['material_anntime_arr']], ["", "timevalue"]),
                create_dict('ETL_evap', [ETL_results['material_evap_thick_arr'], ETL_results['material_evap_rate_arr'], ETL_results['material_evap_pressure_arr']], ["thickness", "rate", "pressure"]),
                create_dict('ETL_cbd_bath', [ETL_results['material_cbd_temp_arr'], ETL_results['material_cbd_time_arr']],["temperature", "time"]),
                create_dict('ETL_cbd_solution',[ETL_results['material_cbd_arr'], ETL_results['material_cbd_quantity_arr']], ["","quantity"]),                
                create_dict('ETL_doping', [ETL_doping_arr, ETL_doping_volume_arr], ["", "volume"]),
                create_dict('ETL_treatment', [ETL_treatment_results['material_arr'], ETL_treatment_results['material_conc_arr'], ETL_treatment_results['material_quantity_arr']], ["", "conc", "quantity"]),
                create_dict('ETL_treatment_SVT', [ETL_treatment_results['material_SVT_arr'], ETL_treatment_results['material_SVT_volume_arr']], ["", "volume"]),
                create_dict('ETL_treatment_spin', [ETL_treatment_results['material_spin_arr'], ETL_treatment_results['material_spinrpm_arr'], ETL_treatment_results['material_spintime_arr']], ["", "rpm", "timevalue"]),
                create_dict('ETL_treatment_ann', [ETL_treatment_results['material_anntemp_arr'], ETL_treatment_results['material_anntime_arr']], ["", "timevalue"]),
                create_dict('ETL_treatment_evap', [ETL_treatment_results['material_evap_thick_arr'], ETL_treatment_results['material_evap_rate_arr'], ETL_treatment_results['material_evap_pressure_arr']], ["thickness", "rate", "pressure"]),
                create_dict('ETL_treatment_cbd_bath', [ETL_treatment_results['material_cbd_temp_arr'], ETL_treatment_results['material_cbd_time_arr']],["temperature", "time"]),
                create_dict('ETL_treatment_cbd_solution',[ETL_treatment_results['material_cbd_arr'], ETL_treatment_results['material_cbd_quantity_arr']], ["","quantity"]),                                
                create_dict('HTL', [HTL_results['material_arr'], HTL_results['material_conc_arr'], HTL_results['material_quantity_arr']], ["", "conc", "quantity"]),
                create_dict('HTL_SVT', [HTL_results['material_SVT_arr'], HTL_results['material_SVT_volume_arr']], ["", "volume"]),
                create_dict('HTL_spin', [HTL_results['material_spin_arr'], HTL_results['material_spinrpm_arr'], HTL_results['material_spintime_arr']], ["", "rpm", "timevalue"]),
                create_dict('HTL_ann', [HTL_results['material_anntemp_arr'], HTL_results['material_anntime_arr']], ["", "timevalue"]),
                create_dict('HTL_evap', [HTL_results['material_evap_thick_arr'], HTL_results['material_evap_rate_arr'], HTL_results['material_evap_pressure_arr']], ["thickness", "rate", "pressure"]),
                create_dict('HTL_cbd_bath', [HTL_results['material_cbd_temp_arr'], HTL_results['material_cbd_time_arr']],["temperature", "time"]),
                create_dict('HTL_cbd_solution',[HTL_results['material_cbd_arr'], HTL_results['material_cbd_quantity_arr']], ["","quantity"]),                  
                create_dict('HTL_doping', [HTL_doping_arr, HTL_doping_volume_arr], ["", "volume"]),
                create_dict('HTL_treatment', [HTL_treatment_results['material_arr'], HTL_treatment_results['material_conc_arr'], HTL_treatment_results['material_quantity_arr']], ["", "conc", "quantity"]),
                create_dict('HTL_treatment_SVT', [HTL_treatment_results['material_SVT_arr'], HTL_treatment_results['material_SVT_volume_arr']], ["", "volume"]),
                create_dict('HTL_treatment_spin', [HTL_treatment_results['material_spin_arr'], HTL_treatment_results['material_spinrpm_arr'], HTL_treatment_results['material_spintime_arr']], ["", "rpm", "timevalue"]),
                create_dict('HTL_treatment_ann', [HTL_treatment_results['material_anntemp_arr'], HTL_treatment_results['material_anntime_arr']], ["", "timevalue"]),
                create_dict('HTL_treatment_evap', [ETL_treatment_results['material_evap_thick_arr'], ETL_treatment_results['material_evap_rate_arr'], ETL_treatment_results['material_evap_pressure_arr']], ["thickness", "rate", "pressure"]),
                create_dict('HTL_treatment_cbd_bath', [HTL_treatment_results['material_cbd_temp_arr'], HTL_treatment_results['material_cbd_time_arr']],["temperature", "time"]),
                create_dict('HTL_treatment_cbd_solution',[HTL_treatment_results['material_cbd_arr'], HTL_treatment_results['material_cbd_quantity_arr']], ["","quantity"]),                  
                create_dict('PV_STR', [PV_STR_results['material_arr'], PV_STR_results['material_conc_arr'], PV_STR_results['material_quantity_arr']], ["", "conc", "quantity"]),
                create_dict('PV_STR_SVT', [PV_STR_results['material_SVT_arr'], PV_STR_results['material_SVT_volume_arr']], ["", "volume"]),
                create_dict('PV_STR_spin', [PV_STR_results['material_spin_arr'], PV_STR_results['material_spinrpm_arr'], PV_STR_results['material_spintime_arr']], ["", "rpm", "timevalue"]),
                create_dict('PV_STR_ann', [PV_STR_results['material_anntemp_arr'], PV_STR_results['material_anntime_arr']], ["", "timevalue"]),
                create_dict('PV_STR_evap', [PV_STR_results['material_evap_thick_arr'], PV_STR_results['material_evap_rate_arr'], PV_STR_results['material_evap_pressure_arr']], ["thickness", "rate", "pressure"]),
                create_dict('PV_STR_cbd_bath', [PV_STR_results['material_cbd_temp_arr'], PV_STR_results['material_cbd_time_arr']],["temperature", "time"]),
                create_dict('PV_STR_cbd_solution',[PV_STR_results['material_cbd_arr'], PV_STR_results['material_cbd_quantity_arr']], ["","quantity"]),   
                create_dict('A_site', [PV_results['a_site_arr'], PV_results['a_site_ratio_arr']], ["", "ratio"]),
                create_dict('B_site', [PV_results['b_site_arr'], PV_results['b_site_ratio_arr']], ["", "ratio"]),
                create_dict('X_site', [PV_results['x_site_arr'], PV_results['x_site_ratio_arr']], ["", "ratio"]),
                create_dict('PV_SVT', [PV_results['material_SVT_arr'], PV_results['material_SVT_volume_arr']], ["", "volume"]),
                create_dict('PV_spin', [PV_results['material_spin_arr'], PV_results['material_spinrpm_arr'], PV_results['material_spintime_arr']], ["", "rpm", "timevalue"]),
                create_dict('PV_ann', [PV_results['material_anntemp_arr'], PV_results['material_anntime_arr']], ["", "timevalue"]),
                create_dict('PV_evap', [PV_results['material_evap_thick_arr'], PV_results['material_evap_rate_arr'], PV_results['material_evap_pressure_arr']], ["thickness", "rate", "pressure"]),
                create_dict('PV_atmos', [PV_results['material_atmostype_arr'], PV_results['material_atmostemp_arr'], PV_results['material_atmoshumidity_arr']], ["", "temperature", "humidity"]),
                create_dict('PV_ADD', [PV_results['material_additive_arr'], PV_results['material_additive_SVT_arr'], PV_results['material_additive_conc_arr'], PV_results['material_additive_conc_unit_arr'],PV_results['material_additive_quantity_arr']], \
                            ["", "SVT", "conc", "conc_unit", "quantity"]),
                create_dict('PV_ASV', [PV_results['material_antisolvent_arr'], PV_results['material_antisolvent_quantity_arr'], PV_results['material_antisolvent_dripping_time_arr']], ["", "quantity", "dripping_time"]),
                create_dict('PV_method', [PV_results['PV_method_arr']], [""]),
                create_dict('PV_molarity', [PV_results['PV_molarity_arr']], [""])                
            ]


            
            
            max_lengths = {}
            prefixes = ['BL', 'BL_SVT', 'BL_spin', 'BL_ann', 'BL_evap', 'BL_cbd_bath', 'BL_cbd_solution', 'ETL', 'ETL_SVT', 'ETL_spin', 'ETL_ann', 'ETL_evap', 'ETL_cbd_bath','ETL_cbd_solution', 'ETL_doping', \
                        'ETL_treatment', 'ETL_treatment_SVT', 'ETL_treatment_spin', 'ETL_treatment_ann', 'ETL_treatment_evap', 'ETL_treatment_cbd_bath', 'ETL_treatment_cbd_solution',\
                        'HTL', 'HTL_SVT', 'HTL_spin', 'HTL_ann', 'HTL_evap', 'HTL_cbd_bath', 'HTL_cbd_solution', 'HTL_doping',\
                        'HTL_treatment', 'HTL_treatment_SVT', 'HTL_treatment_spin', 'HTL_treatment_ann', 'HTL_treatment_evap', 'HTL_treatment_cbd_bath', 'HTL_treatment_cbd_solution',\
                        'PV_STR', 'PV_STR_SVT', 'PV_STR_spin', 'PV_STR_ann', 'PV_STR_evap', 'PV_STR_cbd_bath', 'PV_STR_cbd_solution',\
                       'A_site','B_site','X_site', 'PV_SVT', 'PV_spin', 'PV_ann', 'PV_evap', 'PV_atmos', 'PV_ADD', 'PV_ASV', 'PV_method', 'PV_molarity']
            process_arrays(array_dicts, prefixes, new_row, rm_list, max_lengths)
            update_max_lists(max_lengths, max_lists)

            rows.append(new_row)

print(Err_num)   

BL_max = max_lists.get('BL', 0)
BL_SVT_max = max_lists.get('BL_SVT', 0)
BL_spin_max = max_lists.get('BL_spin', 0)
BL_ann_max = max_lists.get('BL_ann', 0)
BL_evap_max = max_lists.get('BL_evap', 0)
BL_cbd_bath_max = max_lists.get('BL_cbd_bath', 0)
BL_cbd_solution_max = max_lists.get('BL_cbd_solution', 0)

ETL_max = max_lists.get('ETL', 0)

ETL_SVT_max = max_lists.get('ETL_SVT', 0)
ETL_spin_max = max_lists.get('ETL_spin', 0)
ETL_ann_max = max_lists.get('ETL_ann', 0)
ETL_evap_max = max_lists.get('ETL_evap', 0)
ETL_cbd_bath_max = max_lists.get('ETL_cbd_bath', 0)
ETL_cbd_solution_max = max_lists.get('ETL_cbd_solution', 0)
ETL_doping_max = max_lists.get('ETL_doping', 0)

ETL_treatment_max = max_lists.get('ETL_treatment', 0)
ETL_treatment_SVT_max = max_lists.get('ETL_treatment_SVT', 0)
ETL_treatment_spin_max = max_lists.get('ETL_treatment_spin', 0)
ETL_treatment_ann_max = max_lists.get('ETL_treatment_ann', 0)
ETL_treatment_evap_max = max_lists.get('ETL_treatment_evap', 0)
ETL_treatment_cbd_bath_max = max_lists.get('ETL_treatment_cbd_bath', 0)
ETL_treatment_cbd_solution_max = max_lists.get('ETL_treatment_cbd_solution', 0)

HTL_max = max_lists.get('HTL', 0)
HTL_SVT_max = max_lists.get('HTL_SVT', 0)
HTL_spin_max = max_lists.get('HTL_spin', 0)
HTL_ann_max = max_lists.get('HTL_ann', 0)
HTL_evap_max = max_lists.get('HTL_evap', 0)
HTL_cbd_bath_max = max_lists.get('HTL_cbd_bath', 0)
HTL_cbd_solution_max = max_lists.get('HTL_cbd_solution', 0)
HTL_doping_max = max_lists.get('HTL_doping', 0)

HTL_treatment_max = max_lists.get('HTL_treatment', 0)
HTL_treatment_SVT_max = max_lists.get('HTL_treatment_SVT', 0)
HTL_treatment_spin_max = max_lists.get('HTL_treatment_spin', 0)
HTL_treatment_ann_max = max_lists.get('HTL_treatment_ann', 0)
HTL_treatment_evap_max = max_lists.get('HTL_treatment_evap', 0)
HTL_treatment_cbd_bath_max = max_lists.get('HTL_treatment_cbd_bath', 0)
HTL_treatment_cbd_solution_max = max_lists.get('HTL_treatment_cbd_solution', 0)

PV_STR_max= max_lists.get('PV_STR', 0)
PV_STR_SVT_max = max_lists.get('PV_STR_SVT', 0)
PV_STR_spin_max = max_lists.get('PV_STR_spin', 0)
PV_STR_ann_max = max_lists.get('PV_STR_ann', 0)
PV_STR_evap_max = max_lists.get('PV_STR_evap', 0)
PV_STR_cbd_bath_max = max_lists.get('PV_STR_cbd_bath', 0)
PV_STR_cbd_solution_max = max_lists.get('PV_STR_cbd_solution', 0)

A_site_max = max_lists.get('A_site',0)
B_site_max = max_lists.get('B_site',0)
X_site_max = max_lists.get('X_site',0)

SVT_max = max_lists.get('PV_SVT',0)
PV_spin_max= max_lists.get('PV_spin', 0)
PV_ann_max=max_lists.get('PV_ann',0)
PV_evap_max=max_lists.get('PV_evap', 0)
PV_atmos_max=max_lists.get('PV_atmos', 0)
ADD_max=max_lists.get('PV_ADD', 0)
ASV_max=max_lists.get('PV_ASV',0)
MOL_max=max_lists.get('PV_molarity',0)
Method_max=max_lists.get('PV_method',0)


column = ['id', 'author', 'TCO', 'AND', 'device_type', 'note'] + \
         [item for i in range(1) for item in (f'BL_{i+1}',  f'BL_conc_{i+1}', f'BL_quantity_{i+1}')] +\
         [item for i in range(BL_SVT_max) for item in (f'BL_SVT_{i+1}', f'BL_SVT_volume_{i+1}')] +\
         [item for i in range(BL_spin_max) for item in (f'BL_spin_{i+1}', f'BL_spin_rpm_{i+1}', f'BL_spin_timevalue_{i+1}')] +\
         [item for i in range(BL_ann_max) for item in (f'BL_ann_{i+1}', f'BL_ann_timevalue_{i+1}')] +\
         [item for i in range(BL_evap_max) for item in (f'BL_evap_thickness_{i+1}', f'BL_evap_rate_{i+1}', f'BL_evap_pressure_{i+1}')] +\
         [item for i in range(BL_cbd_bath_max) for item in (f'BL_cbd_bath_temperature_{i+1}', f'BL_cbd_bath_time_{i+1}')] +\
         [item for i in range(BL_cbd_solution_max) for item in (f'BL_cbd_solution_{i+1}', f'BL_cbd_solution_quantity_{i+1}')] +\
         ['BL_2'] +\
         [item for i in range(ETL_max) for item in (f'ETL_{i+1}',  f'ETL_conc_{i+1}', f'ETL_quantity_{i+1}')] +\
         [item for i in range(ETL_SVT_max) for item in (f'ETL_SVT_{i+1}', f'ETL_SVT_volume_{i+1}')] +\
         [item for i in range(ETL_spin_max) for item in (f'ETL_spin_{i+1}', f'ETL_spin_rpm_{i+1}', f'ETL_spin_timevalue_{i+1}')] +\
         [item for i in range(ETL_ann_max) for item in (f'ETL_ann_{i+1}', f'ETL_ann_timevalue_{i+1}')] +\
         [item for i in range(ETL_evap_max) for item in (f'ETL_evap_thickness_{i+1}', f'ETL_evap_rate_{i+1}', f'ETL_evap_pressure_{i+1}')] +\
         [item for i in range(ETL_cbd_bath_max) for item in (f'ETL_cbd_bath_temperature_{i+1}', f'ETL_cbd_bath_time_{i+1}')] +\
         [item for i in range(ETL_cbd_solution_max) for item in (f'ETL_cbd_solution_{i+1}', f'ETL_cbd_solution_quantity_{i+1}')] +\
         [item for i in range(ETL_doping_max) for item in (f'ETL_doping_{i+1}', f'ETL_doping_volume_{i+1}')] +\
         [item for i in range(ETL_treatment_max) for item in (f'ETL_treatment_{i+1}',  f'ETL_treatment_conc_{i+1}', f'ETL_treatment_quantity_{i+1}')] +\
         [item for i in range(ETL_treatment_spin_max) for item in (f'ETL_treatment_spin_{i+1}', f'ETL_treatment_spin_rpm_{i+1}', f'ETL_treatment_spin_timevalue_{i+1}')] +\
         [item for i in range(ETL_treatment_evap_max) for item in (f'ETL_evap_thickness_{i+1}', f'ETL_treatment_evap_rate_{i+1}', f'ETL_treatment_evap_pressure_{i+1}')] +\
         [item for i in range(ETL_treatment_cbd_bath_max) for item in (f'ETL_treatment_cbd_bath_temperature_{i+1}', f'ETL_treatment_cbd_bath_time_{i+1}')] +\
         [item for i in range(ETL_treatment_cbd_solution_max) for item in (f'ETL_treatment_cbd_solution_{i+1}', f'ETL_treatment_cbd_solution_quantity_{i+1}')] +\
         [f'PV_method_{i+1}' for i in range(Method_max)]+\
         [f'PV_molarity_{i+1}' for i in range(MOL_max)]+\
         [item for i in range(A_site_max) for item in (f'A_site_{i+1}',  f'A_site_ratio_{i+1}')] +\
         [item for i in range(B_site_max) for item in (f'B_site_{i+1}',  f'B_site_ratio_{i+1}')] +\
         [item for i in range(X_site_max) for item in (f'X_site_{i+1}',  f'X_site_ratio_{i+1}')] +\
         [item for i in range(SVT_max) for item in (f'PV_SVT_{i+1}', f'PV_SVT_volume_{i+1}')] +\
         [item for i in range(PV_spin_max) for item in (f'PV_spin_{i+1}', f'PV_spin_rpm_{i+1}', f'PV_spin_timevalue_{i+1}')] +\
         [item for i in range(PV_ann_max) for item in (f'PV_ann_{i+1}', f'PV_ann_timevalue_{i+1}')] +\
         [item for i in range(PV_evap_max) for item in (f'PV_evap_thickness_{i+1}', f'PV_evap_rate_{i+1}', f'PV_evap_pressure_{i+1}')] +\
         [item for i in range(PV_atmos_max) for item in (f'PV_atmos_{i+1}', f'PV_atmos_temperature_{i+1}')] +\
         [f'PV_atmos_humidity_{i+1}' for i in range(1)]+\
         [item for i in range(ASV_max) for item in (f'PV_ASV_{i+1}', f'PV_ASV_quantity_{i+1}', f'PV_ASV_dripping_time_{i+1}')] +\
         [item for i in range(ADD_max) for item in (f'PV_ADD_{i+1}', f'PV_ADD_SVT_{i+1}', f'PV_ADD_conc_{i+1}', f'PV_ADD_quantity_{i+1}')] +\
         [item for i in range(PV_STR_max) for item in (f'PV_STR_{i+1}',  f'PV_STR_conc_{i+1}', f'PV_STR_quantity_{i+1}')] +\
         [item for i in range(PV_STR_SVT_max) for item in (f'PV_STR_SVT_{i+1}', f'PV_STR_SVT_volume_{i+1}')] +\
         [item for i in range(PV_STR_spin_max) for item in (f'PV_STR_spin_{i+1}', f'PV_STR_spin_rpm_{i+1}', f'PV_STR_spin_timevalue_{i+1}')] +\
         [item for i in range(PV_STR_ann_max) for item in (f'PV_STR_ann_{i+1}', f'PV_STR_ann_timevalue_{i+1}')] +\
         [item for i in range(PV_STR_evap_max) for item in (f'PV_STR_evap_thickness_{i+1}', f'PV_STR_evap_rate_{i+1}', f'PV_STR_evap_pressure_{i+1}')] +\
         [item for i in range(PV_STR_cbd_bath_max) for item in (f'PV_STR_cbd_bath_temperature_{i+1}', f'PV_STR_cbd_bath_time_{i+1}')] +\
         [item for i in range(PV_STR_cbd_solution_max) for item in (f'PV_STR_cbd_solution_{i+1}', f'PV_STR_cbd_solution_quantity_{i+1}')] +\
         [item for i in range(HTL_max) for item in (f'HTL_{i+1}',  f'HTL_conc_{i+1}', f'HTL_quantity_{i+1}')] +\
         [item for i in range(HTL_SVT_max) for item in (f'HTL_SVT_{i+1}', f'HTL_SVT_volume_{i+1}')] +\
         [item for i in range(HTL_spin_max) for item in (f'HTL_spin_{i+1}', f'HTL_spin_rpm_{i+1}', f'HTL_spin_timevalue_{i+1}')] +\
         [item for i in range(HTL_ann_max) for item in (f'HTL_ann_{i+1}', f'HTL_ann_timevalue_{i+1}')] +\
         [item for i in range(HTL_evap_max) for item in (f'HTL_evap_thickness_{i+1}', f'HTL_evap_rate_{i+1}', f'HTL_evap_pressure_{i+1}')] +\
         [item for i in range(HTL_cbd_bath_max) for item in (f'HTL_cbd_bath_temperature_{i+1}', f'HTL_cbd_bath_time_{i+1}')] +\
         [item for i in range(HTL_cbd_solution_max) for item in (f'HTL_cbd_solution_{i+1}', f'HTL_cbd_solution_quantity_{i+1}')] +\
         [item for i in range(HTL_doping_max) for item in (f'HTL_doping_{i+1}', f'HTL_doping_volume_{i+1}')] +\
         [item for i in range(HTL_treatment_max) for item in (f'HTL_treatment_{i+1}',  f'HTL_treatment_conc_{i+1}', f'HTL_treatment_quantity_{i+1}')] +\
         [item for i in range(HTL_treatment_SVT_max) for item in (f'HTL_treatment_SVT_{i+1}', f'HTL_treatment_SVT_volume_{i+1}')] +\
         [item for i in range(HTL_treatment_spin_max) for item in (f'HTL_treatment_spin_{i+1}', f'HTL_treatment_spin_rpm_{i+1}', f'HTL_treatment_spin_timevalue_{i+1}')] +\
         [item for i in range(HTL_treatment_evap_max) for item in (f'HTL_evap_thickness_{i+1}', f'HTL_treatment_evap_rate_{i+1}', f'HTL_treatment_evap_pressure_{i+1}')] +\
         [item for i in range(HTL_treatment_cbd_bath_max) for item in (f'HTL_treatment_cbd_bath_temperature_{i+1}', f'HTL_treatment_cbd_bath_time_{i+1}')] +\
         [item for i in range(HTL_treatment_cbd_solution_max) for item in (f'HTL_treatment_cbd_solution_{i+1}', f'HTL_treatment_cbd_solution_quantity_{i+1}')] +\
         ['Ef_max','FF_max','JS_max','V0_max', 'temperature', 'humidity', 'cell_dimension']
#        [item for i in range(ETL_treatment_SVT_max) for item in (f'ETL_treatment_SVT_{i+1}')]+\, f'ETL_treatment_SVT_volume_{i+1}')] +\

df = pd.DataFrame(rows)
df = df[column]

df_reordered = df.reindex(columns=column)
df_reordered.to_csv('test.csv', encoding='utf-8-sig')


KRICT-SC-00002 김범수 TCO 0 ETL 3 PV 2 HTL 1 Anode 5 device 100
KRICT-SC-00003 김범수 TCO 0 ETL 3 PV 2 HTL 1 Anode 5 device 100
additive old version KRICT-SC-00004
additive old version KRICT-SC-00004
additive old version KRICT-SC-00004
additive old version KRICT-SC-00006
additive old version KRICT-SC-00006
additive old version KRICT-SC-00006
additive old version KRICT-SC-00008
additive old version KRICT-SC-00008
additive old version KRICT-SC-00008
additive old version KRICT-SC-00010
additive old version KRICT-SC-00010
additive old version KRICT-SC-00010
additive old version KRICT-SC-00012
additive old version KRICT-SC-00012
additive old version KRICT-SC-00012
additive old version KRICT-SC-00013
additive old version KRICT-SC-00013
additive old version KRICT-SC-00013
additive old version KRICT-SC-00014
additive old version KRICT-SC-00014
additive old version KRICT-SC-00014
additive old version KRICT-SC-00015
additive old version KRICT-SC-00015
additive old version KRICT-SC-00015
additive old v